# Moonshot 5 Hackathon - Drought in the Netherlands
Using RA2CE to assess the capacity of the road network between Rotterdam and the Ruhr area.

In [19]:
from ra2ce.ra2ce_handler import Ra2ceHandler # import the ra2cehandler to run ra2ce analyses
from pathlib import Path
import networkx as nx
import pandas as pd
from ra2ce.common.io.readers.graph_pickle_reader import GraphPickleReader
from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter

_network_ini_name = "network.ini" # set the name for the network.ini settings file

folder_dir = Path(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od")

network_ini = folder_dir / _network_ini_name
assert network_ini.is_file() # check whether the network.ini file exists

Download the road network from OSM.

In [20]:
race = Ra2ceHandler(network=network_ini, analysis=None)
race.configure()

2023-09-20 02:51:22 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-20 02:51:22 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-20 02:51:22 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\base_graph.p.
2023-09-20 02:51:22 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\base_graph.p.
2023-09-20 02:51:22 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\base_network.feather.
2023-09-20 02:51:22 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: C:\Users\groen_fe\OneDrive - Stichting Deltar

We can inspect the created graph

In [21]:
od_graph = folder_dir / "static" / "output_graph" / "origins_destinations_graph.p"
graph = GraphPickleReader().read(od_graph)

We pick the lowest number of lanes per road segment and assign 1 lane to the road segments without data.

In [22]:
for e in graph.edges(data=True, keys=True):
    if "lanes" not in e[-1]:
        nr_lanes = 1
    else:
        nr_lanes = e[-1]["lanes"]
    if isinstance(nr_lanes, list):
        nr_lanes = min([int(x) for x in nr_lanes])
    if isinstance(nr_lanes, str):
        nr_lanes = int(nr_lanes)
    graph[e[0]][e[1]][e[2]]["lanes_new"] = nr_lanes


We translate the number of lanes to capacity.

In [23]:
capacity = folder_dir / "road_capacity.csv"
capacity = pd.read_csv(capacity)
single_cap_list = capacity["E (pc/h/ln)"].to_list()

for e in graph.edges(data=True, keys=True):
    if e[-1]['avgspeed'] >= 91:
        E = single_cap_list[0]
    elif e[-1]['avgspeed'] < 91 and e[-1]['avgspeed'] > 84:
        E = single_cap_list[1]
    elif e[-1]['avgspeed'] <= 84 and  e[-1]['avgspeed'] > 76:
        E = single_cap_list[2]
    elif e[-1]['avgspeed'] <= 76:
        E = single_cap_list[3]  
    graph[e[0]][e[1]][e[2]]["capacity"] = E * e[-1]["lanes_new"]

In [24]:
exporter = MultiGraphNetworkExporter(basename="updated_network", export_types=["shp", "pickle"])
exporter.export_to_shp(output_dir=folder_dir / "static" / "output_graph", export_data=graph)
exporter.export_to_pickle(output_dir=folder_dir / "static" / "output_graph", export_data=graph)

2023-09-20 02:51:33 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\updated_network_nodes.gpkg
2023-09-20 02:51:33 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\updated_network_nodes.gpkg
2023-09-20 02:51:33 PM - [networks_utils.py:1211] - root - INFO - Saving edges as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\updated_network_edges.gpkg
2023-09-20 02:51:33 PM - [networks_utils.py:1211] - root - INFO - Saving edges as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\output_graph\updated_network_edges.gpkg
2023-09-20 02:51:35 PM - [multi_graph_network_exporter.py:48] - root - INFO - Saved updated_network_

With the network we can start the analysis with looking at a base scenario

In [25]:
from shapely.geometry import LineString, MultiLineString
import geopandas as gpd

In [26]:
origins = gpd.read_file(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\network\origins.gpkg")
destinations = gpd.read_file(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\static\network\destinations.gpkg")

In [27]:
# Rename the origins and destinations
for n in graph.nodes(data=True):
    if "od_id" in n[-1]:
        if "offloading" in n[-1]["od_id"]:
            # the node is an origin
            graph.nodes[n[0]]["od_id"] = origins.loc[origins["identifier"] == int(n[-1]["od_id"].split("_")[-1])]["name"].values[0]
        elif "destination" in n[-1]["od_id"]:
            # the node is a destination
            graph.nodes[n[0]]["od_id"] = destinations.loc[destinations["identifier"] == int(n[-1]["od_id"].split("_")[-1]), "name"].values[0]

In [39]:
list_ods = [(n, v["od_id"]) for n, v in graph.nodes(data=True) if "od_id" in v]
list_destinations = [x for x in list_ods if x[1].startswith("DE")]
list_origins = [x for x in list_ods if x[1].startswith("NL")]

In [32]:
# Filter the OD matrix
full_od_matrix = pd.read_csv(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\full_graph\od_matrix.csv")
full_od_matrix.columns

Index(['UNLO_herkomst', 'UNLO_bestemming', 'v38_Vervoerd_gewicht_sum',
       'v38_Vervoerd_gewicht_count', 'v30_4_Containers_TEU_S_sum',
       'v30_4_Containers_TEU_S_count', 'count_count'],
      dtype='object')

In [35]:
small_od_matrix = full_od_matrix.loc[full_od_matrix["UNLO_bestemming"].isin(list_ods)]

# Save to csv
small_od_matrix.to_csv(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\od_matrix_small.csv", index=False)

In [43]:
weighing = "length"

(
    o_node_list,
    d_node_list,
    origin_list,
    destination_list,
    opt_path_list,
    weighing_list,
    match_ids_list,
    geometries_list,
) = ([], [], [], [], [], [], [], [])
for o in list_origins:
    for d in list_destinations:
        if nx.has_path(graph, o[0], d[0]):
            # calculate the length of the preferred route
            pref_route = nx.dijkstra_path_length(graph, o[0], d[0], weight=weighing)

            # save preferred route nodes
            pref_nodes = nx.dijkstra_path(graph, o[0], d[0], weight=weighing)

            # found out which edges belong to the preferred path
            edgesinpath = list(zip(pref_nodes[0:], pref_nodes[1:]))

            pref_edges = []
            match_list = []
            for u, v in edgesinpath:
                # get edge with the lowest weighing if there are multiple edges that connect u and v
                _uv_graph = graph[u][v]
                edge_key = sorted(
                    _uv_graph, key=lambda x, _fgraph=_uv_graph: _fgraph[x][weighing]
                )[0]
                _uv_graph_edge = _uv_graph[edge_key]
                if "geometry" in _uv_graph_edge:
                    pref_edges.append(_uv_graph_edge["geometry"])
                else:
                    pref_edges.append(
                        LineString(
                            [graph.nodes[u]["geometry"], graph.nodes[v]["geometry"]]
                        )
                    )
                if "rfid" in _uv_graph_edge:
                    match_list.append(_uv_graph_edge["rfid"])

            # compile the road segments into one geometry
            pref_edges = MultiLineString(pref_edges)

            # save all data to lists (of lists)
            o_node_list.append(o[0])
            d_node_list.append(d[0])
            origin_list.append(o[1])
            destination_list.append(d[1])
            opt_path_list.append(pref_nodes)
            weighing_list.append(pref_route)
            match_ids_list.append(match_list)
            geometries_list.append(pref_edges)

# Geodataframe to save all the optimal routes
pref_routes = gpd.GeoDataFrame(
    {
        "o_node": o_node_list,
        "d_node": d_node_list,
        "origin": origin_list,
        "destination": destination_list,
        "opt_path": opt_path_list,
        weighing: weighing_list,
        "match_ids": match_ids_list,
        "geometry": geometries_list,
    },
    geometry="geometry",
    crs="epsg:4326",
)

In [47]:
from ra2ce.analyses.indirect.analyses_indirect import save_gdf
save_gdf(pref_routes, r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\output\optimal_routes.gpkg")

2023-09-20 03:12:02 PM - [analyses_indirect.py:1303] - root - INFO - Results saved to: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\output\optimal_routes.gpkg
2023-09-20 03:12:02 PM - [analyses_indirect.py:1303] - root - INFO - Results saved to: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\small_graph_od\output\optimal_routes.gpkg
